In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import *
import matplotlib.pyplot as plt
def gyh(data):
    s=(data-data.min())/(data.max()-data.min())
    return s

def ycl(path,label):
    data=np.loadtxt(path)
    x_train=[]
    x_test=[]
    
    for i in range(10):
        data_new=data[0+i*100:102400+i*100]
        data_new=data_new.reshape(100,1024)
        np.random.shuffle(data_new)
        train=gyh(data_new[:70,:])
        test=gyh(data_new[70:,:])
        x_train.append(train)
        x_test.append(test)
        
    x_train=np.array(x_train).reshape(-1,1024)
    x_test=np.array(x_test).reshape(-1,1024)
    y_train=np.array([label for i in range(0,700)])
    y_test=np.array([label for i in range(0,300)])
    
    return x_train,y_train,x_test,y_test

def stackkk(a,b,c,d,e,f,g,h):
    aa=np.vstack((a,e))
    bb=np.hstack((b,f))
    cc=np.vstack((c,g))
    dd=np.hstack((d,h))
    return aa,bb,cc,dd

def dataset():
    '''
    0:正常轴承
    1：内圈受损
    2：滚珠受损
    3：外圈受损
    '''
    x_train1,y_train1,x_test1,y_test1=ycl('./12K_21_2/Normal_2_de.txt',0)
    x_train2,y_train2,x_test2,y_test2=ycl('./12K_21_2/IR021_2_de.txt',1)
    x_train3,y_train3,x_test3,y_test3=ycl('./12K_21_2/B021_2_de.txt',2) 
    x_train4,y_train4,x_test4,y_test4=ycl('./12K_21_2/OR021_2_6_de.txt',3)
    
    x_tr1,y_tr1,x_te1,y_te1=stackkk(x_train1,y_train1,x_test1,y_test1,x_train2,y_train2,x_test2,y_test2)
    x_tr2,y_tr2,x_te2,y_te2=stackkk(x_tr1,y_tr1,x_te1,y_te1,x_train3,y_train3,x_test3,y_test3)
    x_tr3,y_tr3,x_te3,y_te3=stackkk(x_tr2,y_tr2,x_te2,y_te2,x_train4,y_train4,x_test4,y_test4)
    
    return x_tr3,y_tr3,x_te3,y_te3
x_train,y_train,x_test,y_test=dataset()

OSError: ./12K_21_2/Normal_2_de.txt not found.

In [3]:

def gyh(data):
    s=(data-data.min())/(data.max()-data.min())
    return s

def ycl(path,label):
    data=np.loadtxt(path)
    x_train=[]
    x_test=[]
    
    for i in range(10):
        data_new=data[0+i*100:102400+i*100]
        data_new=data_new.reshape(100,1024)
        np.random.shuffle(data_new)
        train=gyh(data_new[:70,:])
        test=gyh(data_new[70:,:])
        x_train.append(train)
        x_test.append(test)
        
    x_train=np.array(x_train).reshape(-1,1024)
    x_test=np.array(x_test).reshape(-1,1024)
    y_train=np.array([label for i in range(0,700)])
    y_test=np.array([label for i in range(0,300)])
    
    return x_train,y_train,x_test,y_test

def stackkk(a,b,c,d,e,f,g,h):
    aa=np.vstack((a,e))
    bb=np.hstack((b,f))
    cc=np.vstack((c,g))
    dd=np.hstack((d,h))
    return aa,bb,cc,dd

def dataset():
    '''
    0:正常轴承
    1：内圈受损
    2：滚珠受损
    3：外圈受损
    '''
    x_train1,y_train1,x_test1,y_test1=ycl('./12K_21_2/Normal_2_de.txt',0)
    x_train2,y_train2,x_test2,y_test2=ycl('./12K_21_2/IR021_2_de.txt',1)
    x_train3,y_train3,x_test3,y_test3=ycl('./12K_21_2/B021_2_de.txt',2) 
    x_train4,y_train4,x_test4,y_test4=ycl('./12K_21_2/OR021_2_6_de.txt',3)
    
    x_tr1,y_tr1,x_te1,y_te1=stackkk(x_train1,y_train1,x_test1,y_test1,x_train2,y_train2,x_test2,y_test2)
    x_tr2,y_tr2,x_te2,y_te2=stackkk(x_tr1,y_tr1,x_te1,y_te1,x_train3,y_train3,x_test3,y_test3)
    x_tr3,y_tr3,x_te3,y_te3=stackkk(x_tr2,y_tr2,x_te2,y_te2,x_train4,y_train4,x_test4,y_test4)
    
    return x_tr3,y_tr3,x_te3,y_te3
 

x_train,y_train,x_test,y_test=dataset()

OSError: ./12K_21_2/Normal_2_de.txt not found.

In [4]:
x_train_=x_train.reshape(-1,32,32,1)
x_test_=x_test.reshape(-1,32,32,1)
y_train_=utils.to_categorical(y_train,num_classes=4)
y_test_=utils.to_categorical(y_test,num_classes=4)

NameError: name 'x_train' is not defined

In [5]:
train_db=tf.data.Dataset.from_tensor_slices((x_train_,y_train_))
shuffle_db=train_db.shuffle(1000)# 打乱数据
train_data=shuffle_db.batch(100)# 分组

test_db=tf.data.Dataset.from_tensor_slices((x_test_,y_test_))
test_data=test_db.batch(100)

NameError: name 'x_train_' is not defined

In [6]:
# 取出一个示例
sample=next(iter(train_data))
print('sample:',sample[0].shape,sample[1].shape)

NameError: name 'train_data' is not defined

In [7]:
layers=[
    layers.Input(shape=[32,32,1]),
    layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='SAME',activation='relu'),
    layers.MaxPooling2D(pool_size=2,strides=2),
    #------>[16,16,64]
    
    layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='SAME',activation='relu'),
    layers.MaxPooling2D(pool_size=2,strides=2),
    #------->[8,8,128]
    
    layers.Conv2D(filters=256,kernel_size=3,strides=1,padding='SAME',activation='relu'),
    layers.MaxPooling2D(pool_size=2,strides=2),
    #------->[4,4,256]
    
    layers.Flatten(),
    #-------->[4*4*256]
    
    layers.Dense(1024,activation='relu'),
    layers.Dense(512,activation='relu'),

    layers.Dropout(0.5),
    layers.Dense(4,activation='softmax')
    
    
]

model=keras.Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

In [8]:
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.categorical_crossentropy,
    metrics=[metrics.categorical_accuracy]
)
history=model.fit(train_data,epochs=200,validation_data=test_data)

NameError: name 'train_data' is not defined

In [9]:
# 保存该模型
model.save('./2.model/cnn_1.h5')



OSError: Unable to create file (unable to open file: name = './2.model/cnn_1.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [10]:
#history里有个history属性，是一个字典，
history.history.keys() # 查看字典的keys

NameError: name 'history' is not defined

In [11]:
acc=history.history['categorical_accuracy']
loss=history.history['loss']
val_acc=history.history[ 'val_categorical_accuracy']
val_loss= history.history['val_loss']

NameError: name 'history' is not defined

In [12]:
%matplotlib qt
plt.figure()
plt.xlim(0,200)
plt.ylim(0,1.5)
plt.plot(loss)
plt.plot(acc)
plt.plot(val_loss)
plt.plot(val_acc)
ax=plt.gca()
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')

plt.legend(['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy'],loc=(1,0.5))

NameError: name 'loss' is not defined

In [13]:
weight=model.get_weights()
weight

[array([[[[ 4.34336215e-02, -5.62991090e-02, -4.58460264e-02,
            8.65686685e-02, -8.57909396e-02, -7.87000582e-02,
            2.84805149e-02, -8.07121098e-02,  1.65827796e-02,
            4.28672582e-02,  4.23540175e-02,  8.93995911e-02,
            6.97638541e-02,  2.33618692e-02,  3.46582085e-02,
            5.66301644e-02,  1.81854218e-02, -4.81902733e-02,
            4.82770056e-02,  6.91949129e-02,  1.76550597e-02,
           -2.66007632e-02,  1.28586814e-02, -1.28505975e-02,
           -4.27129418e-02, -5.58572710e-02, -6.79200888e-03,
           -4.93075363e-02, -3.71822044e-02, -2.32413858e-02,
            2.26860866e-02, -1.41976029e-03,  3.74620259e-02,
           -6.40617758e-02,  1.42364949e-03,  2.82552093e-02,
           -9.25093293e-02,  9.07282829e-02, -8.33019465e-02,
            3.89796495e-03, -3.17026675e-03,  8.04333240e-02,
           -2.60334611e-02, -5.27371541e-02, -2.59457901e-02,
           -6.69632703e-02, -8.15091282e-02,  6.31128550e-02,
        

In [14]:
model.predict(x_test_)

NameError: name 'x_test_' is not defined

In [15]:
new_model=keras.models.load_model('./2.model/cnn_1.h5')

OSError: SavedModel file does not exist at: ./2.model/cnn_1.h5/{saved_model.pbtxt|saved_model.pb}

In [16]:
new_model.predict(x_test_)

NameError: name 'new_model' is not defined

In [17]:
pred=new_model.predict(x_test_)
pred_=np.argmax(pred,axis=1)
pred_


NameError: name 'new_model' is not defined

In [18]:
tf.reduce_mean(tf.cast(tf.equal(pred_,np.argmax(y_test_,axis=1)),dtype=tf.float32))

NameError: name 'pred_' is not defined

In [19]:
model2=keras.Sequential([
    keras.layers.Input(shape=[8,128]),
    keras.layers.LSTM(64,dropout=0.5),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(218,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4,activation='softmax')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 512)               33280     
_________________________________________________________________
dense_4 (Dense)              (None, 218)               111834    
_________________________________________________________________
dropout_1 (Dropout)          (None, 218)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 876       
Total params: 195,398
Trainable params: 195,398
Non-trainable params: 0
_________________________________________________________________


In [20]:
x_train2,y_train2,x_test2,y_test2=dataset()

OSError: ./12K_21_2/Normal_2_de.txt not found.

In [21]:
x_train_2=x_train2.reshape(-1,8,128)
x_test_2=x_test2.reshape(-1,8,128)
y_train_2=keras.utils.to_categorical(y_train2,num_classes=4)

NameError: name 'x_train2' is not defined

In [23]:
model2.compile(
    optimizer=optimizers.Adam(),
    loss=losses.categorical_crossentropy,
    metrics=[metrics.categorical_accuracy]
)
model2.fit(x_train_2,y_train_2,epochs=500)

NameError: name 'x_train_2' is not defined

In [24]:
def shellSort(arr): 
    n = len(arr)
    gap = int(n/2)
    while gap > 0: 
        for i in range(gap,n): 
            temp = arr[i] 
            j = i 
            while  j >= gap and arr[j-gap] >temp: 
                arr[j] = arr[j-gap] 
                j -= gap 
            arr[j] = temp 
        gap = int(gap/2)
arr = [ 12, 34, 54, 2, 3] 
n = len(arr) 
print ("排序前:") 
for i in range(n): 
    print(arr[i]), 
shellSort(arr) 
print ("\n排序后:") 
for i in range(n): 
    print(arr[i])

排序前:
12
34
54
2
3

排序后:
2
3
12
34
54


In [25]:
def countSort(arr): 
    output = [0 for i in range(256)] 
    count = [0 for i in range(256)] 
    ans = ["" for _ in arr] 
    for i in arr: 
        count[ord(i)] += 1
    for i in range(256): 
        count[i] += count[i-1] 
    for i in range(len(arr)): 
        output[count[ord(arr[i])]-1] = arr[i] 
        count[ord(arr[i])] -= 1
    for i in range(len(arr)): 
        ans[i] = output[i] 
    return ans  
arr = "wwwrunoobcom"
ans = countSort(arr) 
print ( "字符数组排序 %s"  %("".join(ans)) )

字符数组排序 bcmnoooruwww


In [26]:
def heapify(arr, n, i): 
    largest = i  
    l = 2 * i + 1     # left = 2*i + 1 
    r = 2 * i + 2     # right = 2*i + 2 
    if l < n and arr[i] < arr[l]: 
        largest = l 
    if r < n and arr[largest] < arr[r]: 
        largest = r 
    if largest != i: 
        arr[i],arr[largest] = arr[largest],arr[i]  # 交换
        heapify(arr, n, largest) 
def heapSort(arr): 
    n = len(arr) 
    # Build a maxheap. 
    for i in range(n, -1, -1): 
        heapify(arr, n, i) 
    # 一个个交换元素
    for i in range(n-1, 0, -1): 
        arr[i], arr[0] = arr[0], arr[i]   # 交换
        heapify(arr, i, 0) 
arr = [ 12, 11, 13, 5, 6, 7] 
heapSort(arr) 
n = len(arr) 
print ("排序后") 
for i in range(n): 
    print ("%d" %arr[i])

排序后
5
6
7
11
12
13


In [27]:
class people:
    #定义基本属性
    name = ''
    age = 0
    #定义私有属性,私有属性在类外部无法直接进行访问
    __weight = 0
    #定义构造方法
    def __init__(self,n,a,w):
        self.name = n
        self.age = a
        self.__weight = w
    def speak(self):
        print("%s 说: 我 %d 岁。" %(self.name,self.age))
#单继承示例
class student(people):
    grade = ''
    def __init__(self,n,a,w,g):
        #调用父类的构函
        people.__init__(self,n,a,w)
        self.grade = g
    #覆写父类的方法
    def speak(self):
        print("%s 说: 我 %d 岁了，我在读 %d 年级"%(self.name,self.age,self.grade))
#另一个类，多重继承之前的准备
class speaker():
    topic = ''
    name = ''
    def __init__(self,n,t):
        self.name = n
        self.topic = t
    def speak(self):
        print("我叫 %s，我是一个演说家，我演讲的主题是 %s"%(self.name,self.topic))
#多重继承
class sample(speaker,student):
    a =''
    def __init__(self,n,a,w,g,t):
        student.__init__(self,n,a,w,g)
        speaker.__init__(self,n,t)
test = sample("Tim",25,80,4,"Python")
test.speak()   #方法名同，默认调用的是在括号中排前地父类的方法
class Parent:        # 定义父类
    def myMethod(self):
        print ('调用父类方法')
class Child(Parent): # 定义子类
    def myMethod(self):
        print ('调用子类方法')
c = Child()          # 子类实例
c.myMethod()         # 子类调用重写方法

我叫 Tim，我是一个演说家，我演讲的主题是 Python
调用子类方法


In [28]:
class JustCounter:
    __secretCount = 0  # 私有变量
    publicCount = 0    # 公开变量
    def count(self):
        self.__secretCount += 1
        self.publicCount += 1
        print (self.__secretCount)
counter = JustCounter()
counter.count()
counter.count()
print (counter.publicCount)

1
2
2


In [29]:
class Site:
    def __init__(self, name, url):
        self.name = name       # public
        self.__url = url   # private
    def who(self):
        print('name  : ', self.name)
        print('url : ', self.__url)
    def __foo(self):          # 私有方法
        print('这是私有方法')
    def foo(self):            # 公共方法
        print('这是公共方法')
        self.__foo()
x = Site('菜鸟教程', 'www.runoob.com')
x.who()        # 正常输出
x.foo()        # 正常输出

name  :  菜鸟教程
url :  www.runoob.com
这是公共方法
这是私有方法


In [30]:
class Vector:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __str__(self):
        return 'Vector (%d, %d)' % (self.a, self.b)
    def __add__(self,other):
        return Vector(self.a + other.a, self.b + other.b) 
v1 = Vector(2,10)
v2 = Vector(5,-2)
print (v1 + v2)

Vector (7, 8)


In [31]:
def outer():
    num = 10
    def inner():
        nonlocal num   # nonlocal关键字声明
        num = 100
        print(num)
    inner()
    print(num)
outer()
var_global='global_var'#这个var_global就是全局作用域

100
100
